In [ ]:
from models import get_model
import pandas as pd
import argparse
import pickle
import string
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
import preprocessor as p
from collections import Counter
import os
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix 
from tensorflow.contrib import learn
from tflearn.data_utils import to_categorical, pad_sequences
from scipy import stats
import tflearn
import json

: 

In [4]:
def load_data(filename):
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = [] 
    for i in range(len(data)):
        if(HASH_REMOVE):
            x_text.append(p.tokenize((data[i]['text']).encode('utf-8')))
        else:
            x_text.append(data[i]['text'])
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global NUM_CLASSES, HASH_REMOVE
    if(dataset=="twitter"):
        NUM_CLASSES = 3
        HASH_REMOVE = True
        filename = "data/twitter_data.pkl"
    elif(dataset=="formspring"):
        NUM_CLASSES = 2
        filename = "data/formspring_data.pkl"
    elif(dataset=="wiki"):
        NUM_CLASSES = 2
        filename = "data/wiki_data.pkl"
    return filename

In [5]:
def get_embedding_weights(filename, sep):
    embed_dict = {}
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(sep)
        embed_dict[row[0]] = row[1:]
    print('Loaded from file: ' + str(filename))
    file.close()
    return embed_dict

def map_embedding_weights(embed, vocab, embed_size):
    vocab_size = len(vocab)
    embeddingWeights = np.zeros((vocab_size , embed_size))
    n = 0
    words_missed = []
    for k, v in vocab.iteritems():
        try:
            embeddingWeights[v] = embed[k]
        except:
            n += 1
            words_missed.append(k)
            pass
    print("%d embedding missed"%n, " of " , vocab_size)
    return embeddingWeights

def get_embeddings_dict(vector_type, emb_dim):
    if vector_type == 'sswe':
        emb_dim==50
        sep = '\t'
        vector_file = 'word_vectors/sswe-u.txt'
    elif vector_type =="glove":
        sep = ' '
        if data == "wiki":
            vector_file = 'word_vectors/glove.6B.' + str(emb_dim) + 'd.txt'
        else:
            vector_file = 'word_vectors/glove.twitter.27B.' + str(emb_dim) + 'd.txt'
    else:
        print "ERROR: Please specify a correst model or SSWE cannot be loaded with embed size of: " + str(emb_dim) 
        return None
    
    embed = get_embedding_weights(vector_file, sep)
    return embed

In [6]:
def evaluate_model(model, testX, testY, dump_results, dump_results_file_name):
    temp = model.predict(testX)
    y_pred  = np.argmax(temp, 1)
    y_true = np.argmax(testY, 1)
    if dump_results:
        pd.DataFrame(data={
            "y_true": y_true,
            "y_pred": y_pred
        }).to_csv(dump_results_file_name)
        print("Writter results to \"" + dump_results_file_name + "\"")
    if(data == "twitter"):
        precision = metrics.precision_score(y_true, y_pred, average=None, labels=[0, 2, 1])
        recall = metrics.recall_score(y_true, y_pred, average=None, labels=[0, 2, 1])
        f1_score = metrics.f1_score(y_true, y_pred, average=None, labels=[0, 2, 1])
    else:
        precision = metrics.precision_score(y_true, y_pred, average=None)
        recall = metrics.recall_score(y_true, y_pred, average=None)
        f1_score = metrics.f1_score(y_true, y_pred, average=None)
    print("Precision: " + str(precision) + "\n")
    print("Recall: " + str(recall) + "\n")
    print("f1_score: " + str(f1_score) + "\n")
    print(confusion_matrix(y_true, y_pred))
    print(":: Classification Report")
    print(classification_report(y_true, y_pred))
    return precision, recall, f1_score

In [7]:
def dump_learned_embedding(data, model_type, vector_type, embed_size, embed, vocab_processor):
    vocab = vocab_processor.vocabulary_._mapping
    vocab_size = len(vocab)
    embedDict = {}
    n = 0
    words_missed = []
    for k, v in vocab.iteritems():
        try:
            embeddingDict[v] = embed[k]
        except:
            n += 1
            words_missed.append(k)
            pass
    print("%d embedding missed"%n, " of " , vocab_size)
    
    filename = output_folder_name + data + "_" + model_type + "_" + vector_type + "_" + embed_size + ".pkl"
    with open(filename, 'wb') as handle:
        pickle.dump(embedDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
def get_train_test(data, x_text, labels, oversampling_rate):
    
    X_train, X_test, Y_train, Y_test = train_test_split( x_text, labels, random_state=42, test_size=0.10)

    X_train, Y_train = do_oversampling(data, oversampling_rate, Y_train, X_train)
    
    post_length = np.array([len(x.split(" ")) for x in x_text])
    if(data != "twitter"):
        max_document_length = int(np.percentile(post_length, 95))
    else:
        max_document_length = max(post_length)
    print("Document length : " + str(max_document_length))
    
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length, MAX_FEATURES)
    vocab_processor = vocab_processor.fit(x_text)

    trainX = np.array(list(vocab_processor.transform(X_train)))
    testX = np.array(list(vocab_processor.transform(X_test)))
    
    trainY = np.asarray(Y_train)
    testY = np.asarray(Y_test)
        
    trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)
    testX = pad_sequences(testX, maxlen=max_document_length, value=0.)

    trainY = to_categorical(trainY, nb_classes=NUM_CLASSES)
    testY = to_categorical(testY, nb_classes=NUM_CLASSES)
    
    data_dict = {
        "data": data,
        "trainX" : trainX,
        "trainY" : trainY,
        "testX" : testX,
        "testY" : testY,
        "vocab_processor" : vocab_processor
    }
    
    return data_dict

In [9]:
def return_data(data_dict):
    return data_dict["data"], data_dict["trainX"], data_dict["trainY"], data_dict["testX"], data_dict["testY"], data_dict["vocab_processor"]

In [10]:
def shuffle_weights(model, weights=None):
    """Randomly permute the weights in `model`, or the given `weights`.
    This is a fast approximation of re-initializing the weights of a model.
    Assumes weights are distributed independently of the dimensions of the weight tensors
      (i.e., the weights have the same distribution along each dimension).
    :param Model model: Modify the weights of the given model.
    :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
      If `None`, permute the model's current weights.
    """
    if weights is None:
        weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    # Faster, but less random: only permutes along the first dimension
    # weights = [np.random.permutation(w) for w in weights]
    model.set_weights(weights)

In [11]:
def train(data_dict, model_type, vector_type, embed_size, dump_results=False, dump_results_file_name=""):

    data, trainX, trainY, testX, testY, vocab_processor = return_data(data_dict)
    
    vocab_size = len(vocab_processor.vocabulary_)
    print("Vocabulary Size: {:d}".format(vocab_size))
    vocab = vocab_processor.vocabulary_._mapping
    
    print("Running Model: " + model_type + " with word vector initiliazed with " + vector_type + " word vectors.")
    model = get_model(model_type, trainX.shape[1], vocab_size, embed_size, NUM_CLASSES, LEARN_RATE)

    if(model_type != "cnn"):
        #This is a very crude bugfix, since this method doesnt exist in the DNN class of tflearn
        initial_weights = model.get_weights()
        shuffle_weights(model, initial_weights)
    
    if(model_type == 'cnn'):
        if(vector_type!="random"):
            print("Word vectors used: " + vector_type)
            embeddingWeights = tflearn.get_layer_variables_by_name('EmbeddingLayer')[0]
            model.set_weights(embeddingWeights, map_embedding_weights(get_embeddings_dict(vector_type, embed_size), vocab, embed_size))
            model.fit(trainX, trainY, n_epoch = EPOCHS, shuffle=True, show_metric=True, batch_size=BATCH_SIZE)
        else:
            model.fit(trainX, trainY, n_epoch = EPOCHS, shuffle=True, show_metric=True, batch_size=BATCH_SIZE)
    else:
        if(vector_type!="random"):
            print("Word vectors used: " + vector_type)
            model.layers[0].set_weights([map_embedding_weights(get_embeddings_dict(vector_type, embed_size), vocab, embed_size)])
            model.fit(trainX, trainY, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, 
                  verbose=1)
        else:
            model.fit(trainX, trainY, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, 
                  verbose=1)
    
    return  evaluate_model(model, testX, testY, dump_results, dump_results_file_name)

In [12]:
def print_scores(precision_scores, recall_scores, f1_scores):
    for i in range(NUM_CLASSES):
        print("\nPrecision Class %d (avg): %0.3f (+/- %0.3f)" % (i, precision_scores[:, i].mean(), precision_scores[:, i].std() * 2))
        print( "\nRecall Class %d (avg): %0.3f (+/- %0.3f)" % (i, recall_scores[:, i].mean(), recall_scores[:, i].std() * 2))
        print( "\nF1 score Class %d (avg): %0.3f (+/- %0.3f)" % (i, f1_scores[:, i].mean(), f1_scores[:, i].std() * 2))

In [13]:
def do_oversampling(data, oversampling_rate, labels, x_text): 
    if(data=="twitter"):
        NUM_CLASSES = 3
        
        racism = [i for i in range(len(labels)) if labels[i]==2]
        sexism = [i for i in range(len(labels)) if labels[i]==1]
        x_text = x_text + [x_text[x] for x in racism]*(oversampling_rate-1)+ [x_text[x] for x in sexism]*(oversampling_rate-1)
        labels = labels + [2 for i in range(len(racism))]*(oversampling_rate-1) + [1 for i in range(len(sexism))]*(oversampling_rate-1)
    else:  
        NUM_CLASSES = 2
        bully = [i for i in range(len(labels)) if labels[i]==1]
        x_text = x_text + [x_text[x] for x in bully]*(oversampling_rate-1)
        labels = list(labels) + [1 for i in range(len(bully))]*(oversampling_rate-1)

    print("Counter after oversampling")
    from collections import Counter
    print(Counter(labels))
    
    #this is never used again so im uncommenting it
    #filter_data = []
    #for text in x_text:
    #    filter_data.append("".join(l for l in text if l not in string.punctuation))
        
    return x_text, labels

In [14]:
models = [ 'cnn', 'lstm', 'blstm', 'blstm_attention']
word_vectors = ["random", "glove" ,"sswe"]
EPOCHS = 10
BATCH_SIZE = 128
MAX_FEATURES = 2
NUM_CLASSES = None
DROPOUT = 0.25
LEARN_RATE = 0.01
HASH_REMOVE = None
output_folder_name = "results/"

In [15]:
def run_model(data, oversampling_rate, model_type, vector_type, embed_size, dump_results=False, dump_results_file_name=""):
    x_text, labels = load_data(get_filename(data)) 
    if data == "twitter":
        #map labels
        dict1 = {'racism':2,'sexism':1,'none':0}
        labels = [dict1[b] for b in labels]
    data_dict = get_train_test(data,  x_text, labels, oversampling_rate)
    precision, recall, f1_score = train(data_dict, model_type, vector_type, embed_size, dump_results, dump_results_file_name)
    return precision, recall, f1_score


In [ ]:
data = "formspring"
model_type = "blstm_attention"
vector_type = "random"

for embed_size in [25, 50, 100, 200]:
    run_model(data, 3, model_type, vector_type, embed_size)

## Reproducing table 4

In [ ]:
data_sets = ["formspring", "twitter", "wiki"]
model_types = ["cnn", "blstm_attention"]
vector_types = ["random", "glove"]
#fixed embed size for faster computation
embed_size = 100
results_table_4 = []
for data in data_sets:
    for oversampling_rate in [1, 3]:
        for vector_type in vector_types:
            for model_type in model_types:
                precision, recall, f1_score = run_model(data, 
                                                        oversampling_rate, 
                                                        model_type, 
                                                        vector_type, 
                                                        embed_size, 
                                                        True, 
                                                        "dumps/%s_%d_%s_%s.csv" % (data, oversampling_rate, vector_type, model_type))
                print "Results done for %s %d with vector type %s and model %s" % (data, oversampling_rate, vector_type, model_type)
                results_table_4.append({
                    "dataset": data,
                    "oversampling_rate": oversampling_rate,
                    "vector_type": vector_type,
                    "model_type": model_type,
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1_score
                })

## Reproducing table 6

In [ ]:
data_sets = ["twitter", "formspring", "wiki"]
model_types = [ 'cnn', 'lstm', 'blstm', 'blstm_attention']
vector_types = ["glove"]
#fixed embed size for faster computation
embed_size = 100
results_table_6 = []
for data_set in data_sets:
    for oversampling_rate in [3]:
        for vector_type in vector_types:
            for model_type in model_types:
                precision, recall, f1_score = run_model(data, 
                                                        oversampling_rate, 
                                                        model_type, 
                                                        vector_type, 
                                                        embed_size, 
                                                        True, 
                                                        "dumps/%s_%d_%s_%s.csv" % (data, oversampling_rate, vector_type, model_type))
                print "Results done for %s %d with vector type %s and model %s" % (data, oversampling_rate, vector_type, model_type)
                results_table_6.append({
                    "dataset": data,
                    "oversampling_rate": oversampling_rate,
                    "vector_type": vector_type,
                    "model_type": model_type,
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1_score
                })